### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

sys.path.append("..")
import helper as hp

In [3]:
df = pd.read_excel("/Users/michaelmoret/Desktop/Externa/data/pulling/24-02-24/24022024.xlsx")
df.head(6)

,Test ID,Batch,Young's Modulus,Break Force,Distance to Break,Toughness,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,MPa/%,g,mm,kg/mm².%,NaN,NaN,NaN,NaN
1,NaN,NaN,Gradient St-Strain 1:2,Break Force,Distance to Break,Area St-Strain 1:3,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Start Batch 20240224_Kerling_bleach_W3_,20240224_Kerling_bleach_W3_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20240224_Kerling_bleach_W3_1,20240224_Kerling_bleach_W3_,41.008,72.628,-25.439,761.211,41.0,38.0,breakend,12.5
5,20240224_Kerling_bleach_W3_2,20240224_Kerling_bleach_W3_,37.503,67.397,-26.987,742.341,35.0,35.0,good,12.5


In [4]:
TYPE = "Break stress"

In [5]:
df = df.rename(columns={'Unnamed: 6': 'Diameter 1', 
                        'Unnamed: 7': 'Diameter 2',
                        'Unnamed: 8': 'Note',
                        'Unnamed: 9': 'Humidity',})
df.head(2)

,Test ID,Batch,Young's Modulus,Break Force,Distance to Break,Toughness,Diameter 1,Diameter 2,Note,Humidity
0,NaN,NaN,MPa/%,g,mm,kg/mm².%,NaN,NaN,NaN,NaN
1,NaN,NaN,Gradient St-Strain 1:2,Break Force,Distance to Break,Area St-Strain 1:3,NaN,NaN,NaN,NaN


In [6]:
df = df.dropna()
df.head(2)

,Test ID,Batch,Young's Modulus,Break Force,Distance to Break,Toughness,Diameter 1,Diameter 2,Note,Humidity
4,20240224_Kerling_bleach_W3_1,20240224_Kerling_bleach_W3_,41.008,72.628,-25.439,761.211,41.0,38.0,breakend,12.5
5,20240224_Kerling_bleach_W3_2,20240224_Kerling_bleach_W3_,37.503,67.397,-26.987,742.341,35.0,35.0,good,12.5


In [7]:
df['Break Force'] = df['Break Force'].astype(float)
df['Young\'s Modulus'] = df['Young\'s Modulus'].astype(float)
df['Distance to Break'] = df['Distance to Break'].astype(float)
df['Toughness'] = df['Toughness'].astype(float)
df['Diameter 1'] = df['Diameter 1'].astype(float)
df['Diameter 2'] = df['Diameter 2'].astype(float)
df['Humidity'] = df['Humidity'].astype(float)

df.head(2)

,Test ID,Batch,Young's Modulus,Break Force,Distance to Break,Toughness,Diameter 1,Diameter 2,Note,Humidity
4,20240224_Kerling_bleach_W3_1,20240224_Kerling_bleach_W3_,41.008,72.628,-25.439,761.211,41.0,38.0,breakend,12.5
5,20240224_Kerling_bleach_W3_2,20240224_Kerling_bleach_W3_,37.503,67.397,-26.987,742.341,35.0,35.0,good,12.5


In [8]:
df = df[df['Test ID'] != 'Average:']
df = df[df['Test ID'] != 'S.D.']
df = df[df['Test ID'] != 'Coef. of Variation']

In [9]:
df['Average'] = ((df['Diameter 1'] + df['Diameter 2']) / 2 )/1000
df['Area'] = math.pi * (df['Average'] / 2)**2

In [10]:
df['Baseline area'] = math.pi * 0.035**2
df['Updated Young\'s Modulus'] = df['Young\'s Modulus'] * (df['Baseline area'] / df['Area'])

In [11]:
df['Break stress'] = df['Break Force'] / df['Area'] / 1000

In [12]:
df.head(2)

,Test ID,Batch,Young's Modulus,Break Force,Distance to Break,Toughness,Diameter 1,Diameter 2,Note,Humidity,Average,Area,Baseline area,Updated Young's Modulus,Break stress
4,20240224_Kerling_bleach_W3_1,20240224_Kerling_bleach_W3_,41.008,72.628,-25.439,761.211,41.0,38.0,breakend,12.5,0.0395,0.001225,0.003848,128.786541,59.267965
5,20240224_Kerling_bleach_W3_2,20240224_Kerling_bleach_W3_,37.503,67.397,-26.987,742.341,35.0,35.0,good,12.5,0.0350,0.000962,0.003848,150.012000,70.051041


In [17]:
unique_values = df['Note'].unique()
print(unique_values)

['breakend' 'good' 'brekend' 'short' 'ok' 'slip' 'almost breakend'
 'short?' 'early' 'ok?' 'weakness' 'early?']


In [13]:
count_by_batch = df.groupby('Batch')['Note'].value_counts()
print(count_by_batch)

Batch                        Note           
20240224_Kerling_bleach_NC_  ok                 47
                             breakend           43
                             early               5
                             almost breakend     3
                             short               1
                             early?              1
20240224_Kerling_bleach_W3_  breakend           48
                             ok                 41
                             good                3
                             slip                2
                             early               2
                             short               1
                             short?              1
                             brekend             1
                             almost breakend     1
20240224_Kerling_bleach_W5_  ok                 44
                             breakend           43
                             early               4
                             slip    